In [1]:
import pandas as pd
import matplotlib as plt
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

/home/ha/anaconda3/envs/self/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/ha/anaconda3/envs/self/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


Device 설정 방법

In [2]:

# GPU 하나
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# GPU 여러개
# torch.cuda.device_count()
"""device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net()
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(net)
model.to(device)"""

"device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')\nmodel = Net()\nif torch.cuda.device_count() > 1:\n    model = nn.DataParallel(net)\nmodel.to(device)"

Transform 설정 방식

In [3]:
transform = transforms.Compose([
    transforms.Resize([244,244]),
    transforms.ToTensor()
])

Pretrained Model 가져오기

In [4]:
import torchvision.models as models
model = models.resnet18(pretrained = True)

/home/ha/anaconda3/envs/self/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/ha/anaconda3/envs/self/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Summary 사용 예시

In [5]:
from torchsummary import summary
model.to(device)
summary(model,input_size=(3,288,288))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 144, 144]           9,408
       BatchNorm2d-2         [-1, 64, 144, 144]             128
              ReLU-3         [-1, 64, 144, 144]               0
         MaxPool2d-4           [-1, 64, 72, 72]               0
            Conv2d-5           [-1, 64, 72, 72]          36,864
       BatchNorm2d-6           [-1, 64, 72, 72]             128
              ReLU-7           [-1, 64, 72, 72]               0
            Conv2d-8           [-1, 64, 72, 72]          36,864
       BatchNorm2d-9           [-1, 64, 72, 72]             128
             ReLU-10           [-1, 64, 72, 72]               0
       BasicBlock-11           [-1, 64, 72, 72]               0
           Conv2d-12           [-1, 64, 72, 72]          36,864
      BatchNorm2d-13           [-1, 64, 72, 72]             128
             ReLU-14           [-1, 64,

In [6]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print('The model has {} trainable parameters'.format(count_parameters(model)))

The model has 11689512 trainable parameters


Forward 함수에서 conv layer에서 fc layer 연결시 (batch_size, W\*H\*C) 형식으로 전환하는 방법

In [7]:
def foward(self, x):
    out = self.features(x)
    # 방법 1.
    # out = out.view(out.size(0),-1) # (batch_size, 나머지:32*53*53)
    # 방법 2.
    #out = torch.flatten(out,1) # 첫번쨰는 batch_size이니까~
    # 방법 3.
    #out = out.view(out.shape[0],-1)
    out = self.fc(out)
    return out